In [ ]:
from pymysql import connect
from elasticsearch import Elasticsearch
import json

In [ ]:
database = connect(host='server.acemap.cn',
                   port=13306,
                   user='ee101_web',
                   password='ee101_web',
                   db='ee101_web',
                   charset='utf8')
cursor = database.cursor()
cursor.execute("SELECT * from paper")
result = cursor.fetchall()

In [ ]:
mappings = {
    "mappings": {
        "properties": {
            "paper_id": {
                "type": "long"
            },
            "title": {
                "type": "text",
                "analyzer": "english"
            },
            "year": {
                "type": "long"
            },
            "authors": {
                "type": "text"
            },
            "keywords": {
                "type": "text",
                "analyzer": "english"
            },
            "venue": {
                "type": "text",
                "analyzer": "english"
            }
        }
    }
}

In [ ]:
es = Elasticsearch()
es.indices.create(index='paper', body=mappings)

In [ ]:
for paper in result:
    info = {
        'paper_id': int(paper[0]),
        'title': paper[1],
        'year': int(paper[2]),
        'authors': ", ".join(eval(paper[3])),
        'keywords': ", ".join(eval(paper[4])),
        'venue': paper[5]
    }
    es.index(index='paper', id=info['paper_id'], body=json.dumps(info, ensure_ascii=False))